# Flight Booking Agents with PydanticAI and Together

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Tools_and_Integrations/Agent_Frameworks/PydanticAI/PydanticAI_Agents.ipynb)

<img src="../../images/pydanticai.png" width="700">

This notebook demonstrates how to use PydanticAI with open source models  to create intelligent agents that work together to:
1. Search for flights: The main agent processes user requests to find flights matching specific criteria
2. Extract flight information: A secondary agent parses flight details from web page text
3. Validate results: The framework ensures flights meet the requested origin, destination, and date
4. Handle seat selection: Another agent interprets user seat requests using natural language

Key PydanticAI concepts shown:
- Strongly typed agents: Using Pydantic models to define expected outputs
- Agent tools: The main agent calls the extraction agent as a tool
- Result validation: Procedural validation ensures results meet requirements
- Message history: Preserving context between interactions
- Usage limits: Preventing excessive API calls

This pattern can be extended to more complex workflows and other domains beyond travel booking.

Adapted from [source](https://ai.pydantic.dev/examples/flight-booking/).

## Installation & Setup

Below we install and import necessary libraries and configure our LLM using Together's API. We're using Llama-3.3-70B.

In [2]:
!pip install -qU pydantic-ai

In [6]:
import datetime, os
from dataclasses import dataclass
from typing import Literal

from pydantic import BaseModel, Field
from rich.prompt import Prompt

from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.messages import ModelMessage
from pydantic_ai.usage import Usage, UsageLimits

from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

# Connect PydanticAI to LLMs on Together
llm = OpenAIModel('meta-llama/Llama-3.3-70B-Instruct-Turbo',
                  provider=OpenAIProvider(
                      base_url="https://api.together.xyz/v1",
                      api_key=os.environ.get("TOGETHER_API_KEY"),
                      ),
                   )


## Data Models
Here we define Pydantic models that provide structure for our application:
- `FlightDetails`: Represents flight information with validation
- `NoFlightFound`: For when no suitable flight exists
- `Deps`: Contains data dependencies needed by our agent (search parameters and data source)

In [ ]:

class FlightDetails(BaseModel):
    """Details of the most suitable flight."""

    flight_number: str
    price: int
    origin: str = Field(description='Three-letter airport code')
    destination: str = Field(description='Three-letter airport code')
    date: datetime.date


class NoFlightFound(BaseModel):
    """When no valid flight is found."""


@dataclass
class Deps:
    web_page_text: str
    req_origin: str
    req_destination: str
    req_date: datetime.date


# This agent is responsible for controlling the flow of the conversation.
# It accepts search parameters and returns either flight details or a "not found" response
search_agent = Agent[Deps, FlightDetails | NoFlightFound](
    model=llm,
    result_type=FlightDetails | NoFlightFound,  # type: ignore
    retries=4,
    system_prompt=(
        'Your job is to find the cheapest flight for the user on the given date. '
    ),
    instrument=True,
)

# This agent is responsible for extracting flight details from web page text.
# It parses the raw text and returns a structured list of flights
extraction_agent = Agent(
    model=llm,
    result_type=list[FlightDetails],
    system_prompt='Extract all the flight details from the given text.',
)


## Agent Tools and Validation
PydanticAI allows us to:
1. Create tools that one agent can call (like how our search agent calls the extraction agent)
2. Add validation logic to ensure results meet our requirements

In [ ]:
@search_agent.tool
async def extract_flights(ctx: RunContext[Deps]) -> list[FlightDetails]:
    """Get details of all flights."""
    # we pass the usage to the search agent so requests within this agent are counted
    result = await extraction_agent.run(ctx.deps.web_page_text, usage=ctx.usage)
    logfire.info('found {flight_count} flights', flight_count=len(result.data))
    return result.data


@search_agent.result_validator
async def validate_result(
    ctx: RunContext[Deps], result: FlightDetails | NoFlightFound
) -> FlightDetails | NoFlightFound:
    """Procedural validation that the flight meets the constraints."""
    if isinstance(result, NoFlightFound):
        return result

    errors: list[str] = []
    if result.origin != ctx.deps.req_origin:
        errors.append(
            f'Flight should have origin {ctx.deps.req_origin}, not {result.origin}'
        )
    if result.destination != ctx.deps.req_destination:
        errors.append(
            f'Flight should have destination {ctx.deps.req_destination}, not {result.destination}'
        )
    if result.date != ctx.deps.req_date:
        errors.append(f'Flight should be on {ctx.deps.req_date}, not {result.date}')

    if errors:
        raise ModelRetry('\n'.join(errors))
    else:
        return result



## Seat Selection
We create a separate agent to handle natural language seat preferences. The agent:
1. Understands statements like "I want a window seat"
2. Converts them into structured data (row and seat letter)
3. Handles ambiguity with a retry mechanism

In [ ]:

class SeatPreference(BaseModel):
    row: int = Field(ge=1, le=30)
    seat: Literal['A', 'B', 'C', 'D', 'E', 'F']


class Failed(BaseModel):
    """Unable to extract a seat selection."""


# This agent is responsible for extracting the user's seat selection
seat_preference_agent = Agent[None, SeatPreference | Failed](
    model=llm,
    result_type=SeatPreference | Failed,  # type: ignore
    system_prompt=(
        "Extract the user's seat preference. "
        'Seats A and F are window seats. '
        'Row 1 is the front row and has extra leg room. '
        'Rows 14, and 20 also have extra leg room. '
    ),
)

## Sample Data
For this demo, we use a static dataset of flights. In a real application, this would be:
- Retrieved from an API
- Scraped from a booking website
- Generated by another AI agent

In [8]:
# Sample flight data with various origin/destination/date combinations
flights_web_page = """
1. Flight SFO-AK123
- Price: $350
- Origin: San Francisco International Airport (SFO)
- Destination: Ted Stevens Anchorage International Airport (ANC)
- Date: January 10, 2025

2. Flight SFO-AK456
- Price: $370
- Origin: San Francisco International Airport (SFO)
- Destination: Fairbanks International Airport (FAI)
- Date: January 10, 2025

3. Flight SFO-AK789
- Price: $400
- Origin: San Francisco International Airport (SFO)
- Destination: Juneau International Airport (JNU)
- Date: January 20, 2025

4. Flight NYC-LA101
- Price: $250
- Origin: San Francisco International Airport (SFO)
- Destination: Ted Stevens Anchorage International Airport (ANC)
- Date: January 10, 2025

5. Flight CHI-MIA202
- Price: $200
- Origin: Chicago O'Hare International Airport (ORD)
- Destination: Miami International Airport (MIA)
- Date: January 12, 2025

6. Flight BOS-SEA303
- Price: $120
- Origin: Boston Logan International Airport (BOS)
- Destination: Ted Stevens Anchorage International Airport (ANC)
- Date: January 12, 2025

7. Flight DFW-DEN404
- Price: $150
- Origin: Dallas/Fort Worth International Airport (DFW)
- Destination: Denver International Airport (DEN)
- Date: January 10, 2025

8. Flight ATL-HOU505
- Price: $180
- Origin: Hartsfield-Jackson Atlanta International Airport (ATL)
- Destination: George Bush Intercontinental Airport (IAH)
- Date: January 10, 2025
"""

# We set usage limits to prevent excessive API calls during development/testing
usage_limits = UsageLimits(request_limit=15)

## Overall Logic/Flow
Our application follows this workflow:
1. Set up search parameters (origin, destination, date)
2. Search for flights meeting the criteria
3. Present results to the user
4. Let user choose to book or continue searching
5. If booking, collect seat preference
6. Complete the purchase

In [ ]:
async def main():
    # Configure our search parameters
    deps = Deps(
        web_page_text=flights_web_page,
        req_origin='SFO',
        req_destination='ANC',
        req_date=datetime.date(2025, 1, 10),
    )
    message_history: list[ModelMessage] | None = None
    usage: Usage = Usage()
    
    # Main application loop - continue until user books or no flights found
    while True:
        # Run the search agent with our parameters
        result = await search_agent.run(
            f'Find me a flight from {deps.req_origin} to {deps.req_destination} on {deps.req_date}',
            deps=deps,
            usage=usage,
            message_history=message_history,
            usage_limits=usage_limits,
        )
        
        # Process the result
        if isinstance(result.data, NoFlightFound):
            print('No flight found')
            break
        else:
            flight = result.data
            print(f'Flight found: {flight}')
            answer = Prompt.ask(
                'Do you want to buy this flight, or keep searching? (buy/*search)',
                choices=['buy', 'search', ''],
                show_choices=False,
            )
            if answer == 'buy':
                # If user wants to book, collect seat preference and complete purchase
                seat = await find_seat(usage)
                await buy_tickets(flight, seat)
                break
            else:
                # Continue searching with context from previous interaction
                # This helps the agent understand we want different results
                message_history = result.all_messages(
                    result_tool_return_content='Please suggest another flight'
                )


async def find_seat(usage: Usage) -> SeatPreference:
    """Function to handle seat selection through natural language."""
    message_history: list[ModelMessage] | None = None
    while True:
        answer = Prompt.ask('What seat would you like?')

        result = await seat_preference_agent.run(
            answer,
            message_history=message_history,
            usage=usage,
            usage_limits=usage_limits,
        )
        if isinstance(result.data, SeatPreference):
            return result.data
        else:
            print('Could not understand seat preference. Please try again.')
            message_history = result.all_messages()


async def buy_tickets(flight_details: FlightDetails, seat: SeatPreference):
    """In a real application, this would connect to a booking API."""
    print(f'Purchasing flight {flight_details=!r} {seat=!r}...')

## Running the Application
Execute the main function to start the booking process. In this demo:
1. The agent will find a flight from SFO to ANC on January 10, 2025
2. You can choose to book or search again
3. If booking, you'll be prompted for a seat preference

In [11]:
import asyncio

await main()

Flight found: flight_number='AA101' price=200 origin='SFO' destination='ANC' date=datetime.date(2025, 1, 10)


Do you want to buy this flight, or keep searching? (buy/*search):

What seat would you like?:

Purchasing flight flight_details=FlightDetails(flight_number='AA101', price=200, origin='SFO', destination='ANC', date=datetime.date(2025, 1, 10)) seat=SeatPreference(row=12, seat='A')...
